# Security Troubleshooting

I have some code I want to put into production. It looks something like below. Let's review the code, and then have AI to take a look to see if it's secure.

I read somewhere online you can execute Python code in a string using an `eval()` function. Let's make a simple command line calculator. But what can go wrong?

In [ ]:
def calculate_expression(user_input):
    # Allows users to input a mathematical expression
    result = eval(user_input)
    return result

# Example usage
expression = input("Enter a math expression (e.g., 2 + 2): ")
print("Result:", calculate_expression(expression))

Turns out executing arbitrary Python code from a user input is not the best idea.

Let's look at this SQLite example. Nothing to see here, folks! But let's see if the AI highlights some problems.

In [ ]:
import sqlite3
import requests

open("company_operations.db", "wb") \
    .write(requests.get("https://github.com/thomasnield/anaconda_intro_to_sql/raw/refs/heads/main/company_operations.db").content)

conn = sqlite3.connect("company_operations.db")
customer_id = input("Enter customer ID: ")

sql = f"SELECT * FROM CUSTOMER WHERE CUSTOMER_ID = {customer_id}"

cursor = conn.cursor()
cursor.execute(sql)

customer = cursor.fetchall()
print(customer)
cursor.close()

Well that's not good. Hackers can hijack the SQL and do damaging things against the database. Let's now take a look at an AI agent I built using OpenAI. Ironically, I am going to use ChatGPT to evaluate how safe this code is to deploy in a web service.

In [ ]:
import openai

class FoodMeasureAgent:
    def __init__(self):

        # Initialize OpenAI client
        self.client = openai.OpenAI(api_key="XcBCz1ATmcq0kCCci3QHXDPXWzdhO2h8")
        self.conversation_history = [
            {"role": "system",
             "content": "Convert each food measurement the user provides into grams. Only present the measurement. For example, If asked for a tablespoon of olive oil, only output '14 grams'"
             }
        ]

    def get_response(self, user_input):
        # Add user message to conversation history
        self.conversation_history.append({"role": "user", "content": user_input})

        # Get response from OpenAI
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",  # You can change to "gpt-4" if you have access
            messages=self.conversation_history,
            max_tokens=1000,
            temperature=0.7
        )

        # Extract the assistant's response
        assistant_response = response.choices[0].message.content.strip()

        # Add assistant's response to conversation history
        self.conversation_history.append({"role": "assistant", "content": assistant_response})

        return assistant_response

# Replace with your actual OpenAI API key
agent = FoodMeasureAgent()
print("Welcome! Type a food measurement and I will convert it into grams. Type 'quit' to exit.")
while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        print("Goodbye!")
        break
    response = agent.get_response(user_input)
    print(f"AI: {response}")